# SU(3) en 1+1 dim

In [51]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import nquad
from tqdm import tqdm
import time
from scipy.stats import unitary_group
from scipy.special import gamma
import vegas

In [4]:
def Phi1(arr):
    alpha1, alpha2, alpha3, phi1, phi2 = arr
    x1 = np.cos(alpha1) * np.sin(phi1)
    x2 = np.sin(alpha1) * np.sin(phi1)
    x3 = np.sin(alpha2) * np.cos(phi1) * np.sin(phi2)
    x4 = np.cos(alpha2) * np.cos(phi1) * np.sin(phi2)
    x5 = np.sin(alpha3) * np.cos(phi1) * np.cos(phi2)
    x6 = np.cos(alpha3) * np.cos(phi1) * np.cos(phi2)
    return np.array([x1,x2,x3,x4,x5,x6])

def Phi1_inv(x1, x2, x3, x4, x5, x6):
    alpha1 = np.arctan2(x2,x1)
    if alpha1 < 0:
        alpha1 += np.pi
    alpha2 = np.arctan2(x3,x4)
    if alpha2 < 0:
        alpha2 += np.pi
    alpha3 = np.arctan2(x5,x6)
    if alpha3 < 0:
        alpha3 += np.pi
    phi1 = np.arcsin(np.sqrt(x1**2 + x2**2))
    if abs(x1**2 + x2**2 - 1) < 1e-5:
        phi2 = 0
    else:
        phi2 = np.arcsin(np.sqrt((x3**2+x4**2)/(1-x1**2-x2**2)))
    return alpha1,alpha2,alpha3,phi1,phi2

Phi1_inv_vec = np.vectorize(Phi1_inv)
    
def Phi(x,y,z,t):
    alpha = x + 1j*y
    beta = z + 1j*t
    return np.array([[alpha, -beta.conjugate()], [beta,alpha.conjugate()]])

def Phi2(alpha1, alpha2, alpha3, phi1, phi2, U):
    arr00 = np.exp(1j*alpha1) * np.cos(phi1)
    arr02 = np.exp(1j*alpha1) * np.sin(phi1)
    arr10 = - np.exp(1j*alpha2) * np.sin(phi1) * np.sin(phi2)
    arr11 = np.exp(-1j*alpha1-1j*alpha3) * np.cos(phi2)
    arr12 = np.exp(1j*alpha2) * np.cos(phi1) * np.sin(phi2)
    arr20 = - np.exp(1j*alpha3) * np.sin(phi1) * np.cos(phi2)
    arr21 = - np.exp(-1j*alpha1-1j*alpha2) * np.sin(phi2)
    arr22 = np.exp(1j*alpha3) * np.cos(phi1) * np.cos(phi2)
    arr0 = np.array([[arr00, 0, arr02],[arr10,arr11,arr12],[arr20,arr21,arr22]])
    arr1 = np.zeros((3,3),dtype=complex)
    arr1[:2,:2] = U
    arr1[2,2] = 1
    return arr0.dot(arr1)

def Phi3(x1, x2, x3, x4, x5, x6, x, y, z, t):
    arr0 = Phi1_inv(x1, x2, x3, x4, x5, x6)
    alpha1, alpha2, alpha3, phi1, phi2 = Phi1_inv(x1, x2, x3, x4, x5, x6)
    U = Phi(x, y, z, t)
    return Phi2(alpha1, alpha2, alpha3, phi1, phi2, U)

def diffeo(p1,p2,p3,p4,p5,t1,t2,t3):
    x1 = np.cos(p1)
    x2 = np.sin(p1) * np.cos(p2)
    x3 = np.sin(p1) * np.sin(p2) * np.cos(p3)
    x4 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.cos(p4)
    x5 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.sin(p4) * np.cos(p5)
    x6 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.sin(p4) * np.sin(p5)
    y1 = np.cos(t1)
    y2 = np.sin(t1) * np.cos(t2)
    y3 = np.sin(t1) * np.sin(t2) * np.cos(t3)
    y4 = np.sin(t1) * np.sin(t2) * np.sin(t3)
    return x1,x2,x3,x4,x5,x6,y1,y2,y3,y4

def jacob(p1,p2,p3,p4,p5,t1,t2,t3):
    arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9 = diffeo(p1,p2,p3,p4,p5,t1,t2,t3)
    return Phi3(arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9)

def jacob_vec(p1,p2,p3,p4,p5,t1,t2,t3):
    x1, x2, x3, x4, x5, x6, y1, y2, y3, y4 = diffeo(p1,p2,p3,p4,p5,t1,t2,t3)
    Phi2_vec = np.vectorize(Phi2)
    alpha1, alpha2, alpha3, phi1, phi2 = Phi1_inv_vec(x1, x2, x3, x4, x5, x6)
    U = Phi(y1, y2, y3, y4)
    return Phi2_vec(alpha1, alpha2, alpha3, phi1, phi2, U)
    
def monte_quado(fvec,n_quad = 1000):
    thetas1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas3 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas4 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas5 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
    phis1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    phis2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    phis3 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
    fX = fvec(jacob_vec(thetas1,thetas2,thetas3,thetas4,thetas5,phis1,phis2,phis3))
    return fX.mean()

n_quad = 1000

thetas1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
thetas2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
thetas3 = np.random.uniform(low=0,high=np.pi,size=n_quad)
thetas4 = np.random.uniform(low=0,high=np.pi,size=n_quad)
thetas5 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
phis1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
phis2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
phis3 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
#jacob_vec(thetas1,thetas2,thetas3,thetas4,thetas5,phis1,phis2,phis3)


arr0 = np.array([1,2])
jacob_vec(arr0,arr0,arr0,arr0,arr0,arr0,arr0,arr0)

TypeError: only length-1 arrays can be converted to Python scalars

In [36]:
def test1(x,y):
    return np.array([[x,y],[y,x]])

test1(thetas1,thetas2)[:,:,0]

array([[1.35053325, 0.5745973 ],
       [0.5745973 , 1.35053325]])

In [46]:
thetas1[0]

0.31265011056258246

In [4]:
def Phi1(alpha1, alpha2, alpha3, phi1, phi2):
    x1 = np.cos(alpha1) * np.sin(phi1)
    x2 = np.sin(alpha1) * np.sin(phi1)
    x3 = np.sin(alpha2) * np.cos(phi1) * np.sin(phi2)
    x4 = np.cos(alpha2) * np.cos(phi1) * np.sin(phi2)
    x5 = np.sin(alpha3) * np.cos(phi1) * np.cos(phi2)
    x6 = np.cos(alpha3) * np.cos(phi1) * np.cos(phi2)
    return x1, x2, x3, x4, x5, x6

def Phi1_inv(x1, x2, x3, x4, x5, x6): 
    n = len(x1)
    eps = 1e-4
    alpha1, alpha2, alpha3 = np.zeros(n), np.zeros(n), np.zeros(n)
    phi1, phi2 = np.zeros(n), np.zeros(n)
    alpha1 = np.arctan2(x2,x1)
    alpha1[alpha1 < 0] += np.pi
    alpha2 = np.arctan2(x3, x4)
    alpha2[alpha2 < 0] += np.pi
    alpha3 = np.arctan2(x5,x6)
    alpha3[alpha3 < 0] += np.pi
    phi1 = np.arcsin(np.sqrt(x1**2 + x2**2))
    phi2[abs(x1**2 + x2**2 - 1) < eps] = 0
    x1_mask = x1[abs(x1**2 + x2**2 - 1) >= eps]
    x2_mask = x2[abs(x1**2 + x2**2 - 1) >= eps]
    x3_mask = x3[abs(x1**2 + x2**2 - 1) >= eps]
    x4_mask = x4[abs(x1**2 + x2**2 - 1) >= eps]
    phi2[abs(x1**2 + x2**2 - 1) >= eps] = np.arcsin(np.sqrt((x3_mask**2+x4_mask**2)/(1-x1_mask**2-x2_mask**2)))
    return alpha1,alpha2,alpha3,phi1,phi2
    
def Phi(x,y,z,t):
    alpha = x + 1j*y
    beta = z + 1j*t
    n = len(alpha)
    return np.array([[alpha[:], -beta[:].conjugate()], [beta[:],alpha[:].conjugate()]]).T

def Phi2(alpha1, alpha2, alpha3, phi1, phi2, U):
    n = len(alpha1)
    arr00 = np.exp(1j*alpha1) * np.cos(phi1)
    arr02 = np.exp(1j*alpha1) * np.sin(phi1)
    arr10 = - np.exp(1j*alpha2) * np.sin(phi1) * np.sin(phi2)
    arr11 = np.exp(-1j*alpha1-1j*alpha3) * np.cos(phi2)
    arr12 = np.exp(1j*alpha2) * np.cos(phi1) * np.sin(phi2)
    arr20 = - np.exp(1j*alpha3) * np.sin(phi1) * np.cos(phi2)
    arr21 = - np.exp(-1j*alpha1-1j*alpha2) * np.sin(phi2)
    arr22 = np.exp(1j*alpha3) * np.cos(phi1) * np.cos(phi2)
    #arr0 = np.zeros((n,3,3))
    zeros = np.zeros(n)
    arr0 = np.array([[arr00[:], zeros[:], arr02[:]],[arr10[:],arr11[:],arr12[:]],[arr20[:],arr21[:],arr22[:]]]).T
    arr1 = np.zeros((n,3,3),dtype=complex)
    arr1[:,:2,:2] = U[:]
    arr1[:,2,2] = 1
    return np.matmul(arr0[:],arr1[:])

def Phi3(x1, x2, x3, x4, x5, x6, x, y, z, t):
    alpha1, alpha2, alpha3, phi1, phi2 = Phi1_inv(x1, x2, x3, x4, x5, x6)
    U = Phi(x, y, z, t)
    return Phi2(alpha1, alpha2, alpha3, phi1, phi2, U)

def diffeo(p1,p2,p3,p4,p5,t1,t2,t3):
    x1 = np.cos(p1)
    x2 = np.sin(p1) * np.cos(p2)
    x3 = np.sin(p1) * np.sin(p2) * np.cos(p3)
    x4 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.cos(p4)
    x5 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.sin(p4) * np.cos(p5)
    x6 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.sin(p4) * np.sin(p5)
    y1 = np.cos(t1)
    y2 = np.sin(t1) * np.cos(t2)
    y3 = np.sin(t1) * np.sin(t2) * np.cos(t3)
    y4 = np.sin(t1) * np.sin(t2) * np.sin(t3)
    return x1,x2,x3,x4,x5,x6,y1,y2,y3,y4

def jacob(p1,p2,p3,p4,p5,t1,t2,t3):
    arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9 = diffeo(p1,p2,p3,p4,p5,t1,t2,t3)
    return Phi3(arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9)
    
def monte_quado(f,n_quad = 100000):
    thetas1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas3 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas4 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas5 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
    phis1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    phis2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    phis3 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
    def fvec(U):
        arr = np.zeros(n_quad,dtype=complex)
        arr[:] = f(U[:].T)
        return arr
    fX = fvec(jacob(thetas1,thetas2,thetas3,thetas4,thetas5,phis1,phis2,phis3)) * np.sin(thetas1)**4 * np.sin(thetas2)**3 * np.sin(thetas3)**2 * np.sin(thetas4) * np.sin(phis1)**2 * np.sin(phis2)
    return 2*np.pi**3 * fX.mean()

def test(U):
    #return U[1,2] * U[2,0] * U[0,1]
    return U[0,0]

def test_vec(U_ls):
    arr = np.zeros(3)
    arr[:] = test(U_ls[:])
    return arr

arr = np.array([[1,2,3],[3,1,5],[4,4,3]])
U_ls = np.array([np.eye(3),np.eye(3),arr])
test_vec(U_ls)

monte_quado(test)

(0.0016768702606170215-0.004621082930834162j)

In [47]:
U = np.eye(3)
np.conj(U.T)[1,1]

1.0

In [6]:
def test(U):
    return U[1,0] * np.conj(U[1,0])

def monte_quado(f,n_quad=100000):
    t1 = np.random.uniform(low=0,high=np.pi,size=3*n_quad)
    t2 = np.random.uniform(low=0,high=np.pi,size=3*n_quad)
    t3 = np.random.uniform(low=0,high=2*np.pi,size=3*n_quad)
    y1 = np.cos(t1)
    y2 = np.sin(t1) * np.cos(t2)
    y3 = np.sin(t1) * np.sin(t2) * np.cos(t3)
    y4 = np.sin(t1) * np.sin(t2) * np.sin(t3)
    arr1 = Phi(y1[:n_quad],y2[:n_quad],y3[:n_quad],y4[:n_quad])
    arr2 = Phi(y1[n_quad:2*n_quad],y2[n_quad:2*n_quad],y3[n_quad:2*n_quad],y4[n_quad:2*n_quad])
    arr3 = Phi(y1[2*n_quad:3*n_quad],y2[2*n_quad:3*n_quad],y3[2*n_quad:3*n_quad],y4[2*n_quad:3*n_quad])
    R, S, T = np.zeros((n_quad,3,3),dtype=complex), np.zeros((n_quad,3,3),dtype=complex), np.zeros((n_quad,3,3),dtype=complex)
    R[:,:2,:2] = arr1[:]
    R[:,2,2] = 1
    zeros = np.zeros(n_quad)
    ones = np.ones(n_quad)
    S[:] = np.array([[arr2[:,0,0], zeros[:], arr2[:,0,1]],[zeros[:], ones[:], zeros[:]],[arr2[:,1,0], zeros[:], arr2[:,1,1]]]).T
    T[:,0,0] = 1
    T[:,1:3,1:3] = arr3[:] 
    X = np.zeros((n_quad,3,3),dtype=complex)
    X[:] = np.matmul(R[:],np.matmul(S[:],T[:]))
    def fvec(U):
        arr = np.zeros(n_quad,dtype=complex)
        arr[:] = f(U[:].T)
        return arr
    return 2*np.pi**3*fvec(X).mean()

monte_quado(test)

(14.622614371144774+0j)

In [8]:
n_quad = 100

t1 = np.random.uniform(low=0,high=np.pi,size=3*n_quad)
t2 = np.random.uniform(low=0,high=np.pi,size=3*n_quad)
t3 = np.random.uniform(low=0,high=2*np.pi,size=3*n_quad)
y1 = np.cos(t1)
y2 = np.sin(t1) * np.cos(t2)
y3 = np.sin(t1) * np.sin(t2) * np.cos(t3)
y4 = np.sin(t1) * np.sin(t2) * np.sin(t3)
arr1 = Phi(y1[:n_quad],y2[:n_quad],y3[:n_quad],y4[:n_quad])
arr2 = Phi(y1[n_quad:2*n_quad],y2[n_quad:2*n_quad],y3[n_quad:2*n_quad],y4[n_quad:2*n_quad])
arr3 = Phi(y1[2*n_quad:3*n_quad],y2[2*n_quad:3*n_quad],y3[2*n_quad:3*n_quad],y4[2*n_quad:3*n_quad])
R, S, T = np.zeros((n_quad,3,3),dtype=complex), np.zeros((n_quad,3,3),dtype=complex), np.zeros((n_quad,3,3),dtype=complex)
R[:,:2,:2] = arr1[:]
R[:,2,2] = 1
zeros = np.zeros(n_quad)
ones = np.ones(n_quad)
S[:] = np.array([[arr2[:,0,0], zeros[:], arr2[:,0,1]],[zeros[:], ones[:], zeros[:]],[arr2[:,1,0], zeros[:], arr2[:,1,1]]]).T
T[:,0,0] =1
T[:,1:3,1:3] = arr3[:] 
X = np.zeros((n_quad,3,3),dtype=complex)
X[:] = np.matmul(R[:],np.matmul(S[:],T[:]))
np.linalg.det(X[10])

(0.9999999999999999+5.551115123125782e-17j)

In [9]:
def monte_quado(f,n_quad = 100000):
    thetas1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas3 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas4 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    thetas5 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
    phis1 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    phis2 = np.random.uniform(low=0,high=np.pi,size=n_quad)
    phis3 = np.random.uniform(low=0,high=2*np.pi,size=n_quad)
    def fvec(U):
        arr = np.zeros(n_quad,dtype=complex)
        arr[:] = f(U[:].T)
        return arr
    fX = fvec(jacob(thetas1,thetas2,thetas3,thetas4,thetas5,phis1,phis2,phis3))
    return fX.mean()

In [10]:
import numpy as np
from scipy.linalg import logm, expm
from scipy.stats import unitary_group

N = 4

def haar_measure_integrate(func, num_samples):
    """
    Perform Monte Carlo integration using the Haar measure on SU(3).
    
    Parameters:
        func (function): The function to be integrated. It takes an SU(3) element as input and returns a real number.
        num_samples (int): The number of samples to use for integration.
    
    Returns:
        (float): The estimated integral value.
    """
    integral = 0.0
    
    x = unitary_group.rvs(N, size=num_samples)
    det_sign = np.sign(np.linalg.det(x))
    sign_correction = (det_sign < 0) * 1j + (det_sign > 0)
    arr = x * sign_correction[:, np.newaxis, np.newaxis]
    lattice = np.zeros((num_samples,N,N),dtype=complex)
    lattice[:] = arr[:]
    
    for i in range(num_samples):
        integral += func(lattice[i])
    integral /= num_samples
    return integral

# Example usage
def my_function(u):
    # Define your function to be integrated here
    # For example, let's calculate the trace of the matrix u
    return np.linalg.det(u)

num_samples = 100000  # Number of samples for Monte Carlo integration
result = haar_measure_integrate(my_function, num_samples)
print("Integral result:", result)

Integral result: (-0.002854802950578576-0.0033412945639819227j)


In [29]:
l1 = np.array([[0,1,0],[1,0,0],[0,0,0]])
l2 = np.array([[0,-1j,0],[1j,0,0],[0,0,0]])
l3 = np.array([[1,0,0],[0,-1,0],[0,0,0]])
l4 = np.array([[0,0,1],[0,0,0],[1,0,0]])
l5 = np.array([[0,0,-1j],[0,0,0],[1j,0,0]])
l6 = np.array([[0,0,0],[0,0,1],[0,1,0]])
l7 = np.array([[0,0,0],[0,0,-1j],[0,1j,0]])
l8 = 1/np.sqrt(3) * np.array([[1,0,0],[0,1,0],[0,0,-2]])

def integrand(f,beta,b,theta,alpha,gamma,a,c,phi):
    fvec = np.vectorize(f)
    dv = np.sin(2*beta) * np.sin(2*b) * np.sin(2*theta) * np.sin(theta)**2
    return fvec(alpha,beta,gamma,theta,a,b,c,phi) * dv

def test(beta,b,theta,alpha,gamma,a,c,phi):
    A = expm(1j*l3*alpha)
    B = expm(1j*l2*beta)
    C = expm(1j*l3*gamma)
    D = expm(1j*l5*theta)
    E = expm(1j*l3*a)
    F = expm(1j*l2*b)
    G = expm(1j*l3*c)
    H = expm(1j*l8*phi)
    U = np.matmul(A,np.matmul(B,np.matmul(C,np.matmul(D,np.matmul(E,np.matmul(F,np.matmul(G,H)))))))
    #return 1
    #return U[1,2] * np.conj(U[1,2])
    return U[1,1] * U[2,2] * U[0,0]

def monte_quado(f,n_samples=10000):
    alpha = np.random.uniform(low=0,high=np.pi,size=n_samples)
    gamma = np.random.uniform(low=0,high=np.pi,size=n_samples)
    a = np.random.uniform(low=0,high=np.pi,size=n_samples)
    c = np.random.uniform(low=0,high=np.pi,size=n_samples)
    beta = np.random.uniform(low=0,high=np.pi/2,size=n_samples)
    b = np.random.uniform(low=0,high=np.pi/2,size=n_samples)
    theta = np.random.uniform(low=0,high=np.pi/2,size=n_samples)
    phi = np.random.uniform(low=0,high=2*np.pi,size=n_samples)
    arr = integrand(f,beta,b,theta,alpha,gamma,a,c,phi)
    return 1/4*np.pi**8 * arr.mean() / 304.15844844637223

monte_quado(test)

(0.258576701456169-0.00013275723854063275j)

In [ ]:
def integrand(f,theta,phi):
    fvec = np.vectorize(f)
    dv = 64 * np.sin(phi/2)**2 * np.sin(1/2*(theta+phi/2))**2 * np.sin(1/2*(theta-phi/2))**2
    return fvec(alpha,beta,gamma,theta,a,b,c,phi) * dv

def test(beta,b,theta,alpha,gamma,a,c,phi):
    A = expm(1j*l3*alpha)
    B = expm(1j*l2*beta)
    C = expm(1j*l3*gamma)
    D = expm(1j*l5*theta)
    E = expm(1j*l3*a)
    F = expm(1j*l2*b)
    G = expm(1j*l3*c)
    H = expm(1j*l8*phi)
    U = np.matmul(A,np.matmul(B,np.matmul(C,np.matmul(D,np.matmul(E,np.matmul(F,np.matmul(G,H)))))))
    #return 1
    #return U[1,2] * np.conj(U[1,2])
    return U[1,1] * U[2,2] * U[0,0]

def monte_quado(f,n_samples=10000):
    theta = np.random.uniform(low=-3*np.pi,high=3*np.pi,size=n_samples)
    phi = np.random.uniform(low=-2*np.pi,high=2*np.pi,size=n_samples)
    arr = integrand(f,theta,phi)
    return arr.mean()

monte_quado(test)

In [2]:
import vegas
import math

def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return math.exp(-dx2 * 100.) * 1013.2118364296088

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   2.0(1.4)        2.0(1.4)            0.00     1.00
  2   1.32(22)        1.34(22)            0.21     0.64
  3   0.786(78)       0.848(74)           2.89     0.06
  4   0.950(69)       0.902(50)           2.27     0.08
  5   1.000(39)       0.964(31)           2.29     0.06
  6   1.034(25)       1.006(19)           2.45     0.03
  7   0.965(18)       0.984(13)           2.44     0.02
  8   1.008(14)       0.9950(97)          2.30     0.02
  9   0.986(13)       0.9918(78)          2.05     0.04
 10   0.987(12)       0.9903(65)          1.84     0.06

result = 0.9903(65)    Q = 0.06


In [50]:
def Phi1(arr):
    alpha1, alpha2, alpha3, phi1, phi2 = arr
    x1 = np.cos(alpha1) * np.sin(phi1)
    x2 = np.sin(alpha1) * np.sin(phi1)
    x3 = np.sin(alpha2) * np.cos(phi1) * np.sin(phi2)
    x4 = np.cos(alpha2) * np.cos(phi1) * np.sin(phi2)
    x5 = np.sin(alpha3) * np.cos(phi1) * np.cos(phi2)
    x6 = np.cos(alpha3) * np.cos(phi1) * np.cos(phi2)
    return np.array([x1,x2,x3,x4,x5,x6])

def Phi1_inv(x1, x2, x3, x4, x5, x6):
    alpha1 = np.arctan2(x2,x1)
    if alpha1 < 0:
        alpha1 += np.pi
    alpha2 = np.arctan2(x3,x4)
    if alpha2 < 0:
        alpha2 += np.pi
    alpha3 = np.arctan2(x5,x6)
    if alpha3 < 0:
        alpha3 += np.pi
    phi1 = np.arcsin(np.sqrt(x1**2 + x2**2))
    if abs(x1**2 + x2**2 - 1) < 1e-5:
        phi2 = 0
    else:
        phi2 = np.arcsin(np.sqrt((x3**2+x4**2)/(1-x1**2-x2**2)))
    return alpha1,alpha2,alpha3,phi1,phi2

Phi1_inv_vec = np.vectorize(Phi1_inv)
    
def Phi(x,y,z,t):
    alpha = x + 1j*y
    beta = z + 1j*t
    return np.array([[alpha, -beta.conjugate()], [beta,alpha.conjugate()]])

def Phi2(alpha1, alpha2, alpha3, phi1, phi2, U):
    arr00 = np.exp(1j*alpha1) * np.cos(phi1)
    arr02 = np.exp(1j*alpha1) * np.sin(phi1)
    arr10 = - np.exp(1j*alpha2) * np.sin(phi1) * np.sin(phi2)
    arr11 = np.exp(-1j*alpha1-1j*alpha3) * np.cos(phi2)
    arr12 = np.exp(1j*alpha2) * np.cos(phi1) * np.sin(phi2)
    arr20 = - np.exp(1j*alpha3) * np.sin(phi1) * np.cos(phi2)
    arr21 = - np.exp(-1j*alpha1-1j*alpha2) * np.sin(phi2)
    arr22 = np.exp(1j*alpha3) * np.cos(phi1) * np.cos(phi2)
    arr0 = np.array([[arr00, 0, arr02],[arr10,arr11,arr12],[arr20,arr21,arr22]])
    arr1 = np.zeros((3,3),dtype=complex)
    arr1[:2,:2] = U
    arr1[2,2] = 1
    return arr0.dot(arr1)

def Phi3(x1, x2, x3, x4, x5, x6, x, y, z, t):
    arr0 = Phi1_inv(x1, x2, x3, x4, x5, x6)
    alpha1, alpha2, alpha3, phi1, phi2 = Phi1_inv(x1, x2, x3, x4, x5, x6)
    U = Phi(x, y, z, t)
    return Phi2(alpha1, alpha2, alpha3, phi1, phi2, U)

def diffeo(p1,p2,p3,p4,p5,t1,t2,t3):
    x1 = np.cos(p1)
    x2 = np.sin(p1) * np.cos(p2)
    x3 = np.sin(p1) * np.sin(p2) * np.cos(p3)
    x4 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.cos(p4)
    x5 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.sin(p4) * np.cos(p5)
    x6 = np.sin(p1) * np.sin(p2) * np.sin(p3) * np.sin(p4) * np.sin(p5)
    y1 = np.cos(t1)
    y2 = np.sin(t1) * np.cos(t2)
    y3 = np.sin(t1) * np.sin(t2) * np.cos(t3)
    y4 = np.sin(t1) * np.sin(t2) * np.sin(t3)
    return x1,x2,x3,x4,x5,x6,y1,y2,y3,y4

def jacob(p1,p2,p3,p4,p5,t1,t2,t3):
    arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9 = diffeo(p1,p2,p3,p4,p5,t1,t2,t3)
    return Phi3(arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9)

def jacob_vec(p1,p2,p3,p4,p5,t1,t2,t3):
    x1, x2, x3, x4, x5, x6, y1, y2, y3, y4 = diffeo(p1,p2,p3,p4,p5,t1,t2,t3)
    Phi2_vec = np.vectorize(Phi2)
    alpha1, alpha2, alpha3, phi1, phi2 = Phi1_inv_vec(x1, x2, x3, x4, x5, x6)
    U = Phi(y1, y2, y3, y4)
    return Phi2_vec(alpha1, alpha2, alpha3, phi1, phi2, U)

def jacob(arr):
    p1,p2,p3,p4,p5,t1,t2,t3 = arr
    arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9 = diffeo(p1,p2,p3,p4,p5,t1,t2,t3)
    integrand = Phi3(arg0, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9)
    return integrand
    

def test(arr):
    M = jacob(arr)
    p1,p2,p3,p4,p5,t1,t2,t3 = arr
    res = M[0,1] * np.conj(M[0,1])
    #res = 1
    #res = M[0,1] * M[1,2] * M[2,0]
    return res/(2*np.pi**5) * np.sin(p1)**4 * np.sin(p2)**3 * np.sin(p3)**2 * np.sin(p4) * np.sin(t1)**2 * np.sin(t2)

ls = []
for _ in range(2):
    ls.append([0,np.pi])
ls.append([0,2*np.pi])
for _ in range(4):
    ls.append([0,np.pi])
ls.append([0,2*np.pi])
    
integ = vegas.Integrator(ls)

result = integ(test, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

/var/folders/c7/_nlmffn91z9864jmkznb7mx9hx_pfj/T/ipykernel_10424/1470771886.py:105: ComplexWarning: Casting complex values to real discards the imaginary part
  result = integ(test, nitn=10, neval=1000)


itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.298(49)       0.298(49)           0.00     1.00
  2   0.337(54)       0.316(36)           0.29     0.59
  3   0.359(48)       0.331(29)           0.39     0.67
  4   0.340(20)       0.337(16)           0.28     0.84
  5   0.315(12)       0.3232(98)          0.50     0.73
  6   0.3394(98)      0.3313(69)          0.67     0.64
  7   0.3323(74)      0.3318(50)          0.56     0.76
  8   0.3373(63)      0.3340(39)          0.55     0.80
  9   0.3284(51)      0.3319(31)          0.57     0.80
 10   0.3259(44)      0.3299(25)          0.65     0.76

result = 0.3299(25)    Q = 0.76


In [59]:
l1 = np.array([[0,1,0],[1,0,0],[0,0,0]])
l2 = np.array([[0,-1j,0],[1j,0,0],[0,0,0]])
l3 = np.array([[1,0,0],[0,-1,0],[0,0,0]])
l4 = np.array([[0,0,1],[0,0,0],[1,0,0]])
l5 = np.array([[0,0,-1j],[0,0,0],[1j,0,0]])
l6 = np.array([[0,0,0],[0,0,1],[0,1,0]])
l7 = np.array([[0,0,0],[0,0,-1j],[0,1j,0]])
l8 = 1/np.sqrt(3) * np.array([[1,0,0],[0,1,0],[0,0,-2]])

def su3_gen(a,b,phi,d,e,f,g,h):
    exponent = a*l1 + b*l2 + c*l3 + d*l4 + e*l5 + f*l6 + g*l7 + h*l8
    return np.exp(1j*exponent/2)



array([[0.70478582+0.70942015j, 1.44688904+0.79043908j,
        1.44688904+0.79043908j],
       [0.53228073+0.29078629j, 0.97775388-0.20975548j,
        1.44688904+0.79043908j],
       [0.53228073+0.29078629j, 0.53228073+0.29078629j,
        0.83791183-0.54580561j]])